In [1]:
import os
import wandb
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
wandb.login()

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anu-mh0206 (anu-mh0206-northeastern-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
# Simple but Customized XGBoost with W&B Tracking
import wandb
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration - Easy to modify these parameters
config = {
    "max_depth": 6,
    "learning_rate": 0.3,
    "n_estimators": 100,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "seed": 42
}

# Initialize W&B
run = wandb.init(
    project="Lab1-visualize-models",
    name="xgboost-simple-custom",
    config=config
)

# Load data
print("Loading dataset...")
data = np.loadtxt('./dermatology.data', delimiter=',',
                  converters={33: lambda x: int(x == '?'), 34: lambda x: int(x) - 1})

# Prepare features and labels
X = data[:, :33]
y = data[:, 34]
n_classes = len(np.unique(y))

print(f"Dataset: {len(X)} samples, {X.shape[1]} features, {n_classes} classes")

# Split data with stratification (better than simple slice)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=config["seed"]
)

# Create XGBoost datasets
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set up parameters
params = {
    'objective': 'multi:softmax',
    'num_class': n_classes,
    'max_depth': config["max_depth"],
    'learning_rate': config["learning_rate"],
    'subsample': config["subsample"],
    'colsample_bytree': config["colsample_bytree"],
    'seed': config["seed"],
    'eval_metric': 'mlogloss'
}

# Train with evaluation
print("\nTraining model...")
watchlist = [(dtrain, 'train'), (dtest, 'eval')]
evals_result = {}

model = xgb.train(
    params,
    dtrain,
    num_boost_round=config["n_estimators"],
    evals=watchlist,
    evals_result=evals_result,
    early_stopping_rounds=10,
    verbose_eval=20
)

# Make predictions
y_train_pred = model.predict(dtrain)
y_test_pred = model.predict(dtest)

# Calculate metrics
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print(f"\nTrain Accuracy: {train_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

# Log to W&B
wandb.log({
    "train_accuracy": train_acc,
    "test_accuracy": test_acc,
    "best_iteration": model.best_iteration
})

# Create and log confusion matrix
cm = confusion_matrix(y_test, y_test_pred)
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
ax.set_title('Confusion Matrix')
ax.set_ylabel('True Label')
ax.set_xlabel('Predicted Label')
plt.tight_layout()

# Log as both image and table
wandb.log({
    "confusion_matrix_plot": wandb.Image(fig),
    "confusion_matrix_table": wandb.Table(data=cm.tolist(), 
                                          columns=[f"Pred_{i}" for i in range(n_classes)])
})
plt.close(fig)

# Feature importance (simple version)
importance = model.get_score(importance_type='weight')
if importance:
    # Get top 10 features
    sorted_imp = sorted(importance.items(), key=lambda x: x[1], reverse=True)[:10]
    
    # Plot
    fig2, ax2 = plt.subplots(figsize=(8, 6))
    features = [f"f{f[0][1:]}" for f in sorted_imp]
    scores = [f[1] for f in sorted_imp]
    
    ax2.barh(features, scores)
    ax2.set_xlabel('Importance (Weight)')
    ax2.set_title('Top 10 Feature Importance')
    plt.tight_layout()
    
    # Log both as image and as table
    wandb.log({
        "feature_importance_plot": wandb.Image(fig2),
        "feature_importance_table": wandb.Table(
            data=[[f, s] for f, s in zip(features, scores)],
            columns=["Feature", "Importance"]
        )
    })
    plt.close(fig2)

# Log learning curves with step tracking
if evals_result:
    fig3, ax3 = plt.subplots(figsize=(10, 5))
    
    # Plot training curves
    epochs = len(evals_result['train']['mlogloss'])
    x_axis = list(range(0, epochs))
    
    ax3.plot(x_axis, evals_result['train']['mlogloss'], label='Train', marker='o', markersize=3)
    ax3.plot(x_axis, evals_result['eval']['mlogloss'], label='Test', marker='s', markersize=3)
    ax3.set_xlabel('Boosting Round')
    ax3.set_ylabel('Log Loss')
    ax3.set_title('Training Progress')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    plt.tight_layout()
    
    wandb.log({"learning_curves_plot": wandb.Image(fig3)})
    plt.close(fig3)
    
    # Also log the curves as line plots in W&B
    for i, (train_loss, eval_loss) in enumerate(zip(evals_result['train']['mlogloss'], 
                                                     evals_result['eval']['mlogloss'])):
        wandb.log({
            "train_loss": train_loss,
            "eval_loss": eval_loss,
            "epoch": i
        })

print("\nAll visualizations logged to W&B dashboard")
print("Check the 'Media' tab in W&B for images")
print("Check the 'Charts' tab for line plots")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

# Save model (simple approach)
try:
    model_path = "model.json"
    model.save_model(model_path)
    
    # Create W&B artifact
    artifact = wandb.Artifact(
        name="xgboost-model",
        type="model"
    )
    artifact.add_file(model_path)
    run.log_artifact(artifact)
    print(f"\nModel saved as W&B artifact")
except Exception as e:
    print(f"Note: Could not save model artifact: {e}")

print(f"\nView run at: {run.url}")
wandb.finish()

Loading dataset...
Dataset: 366 samples, 33 features, 6 classes

Training model...
[0]	train-mlogloss:1.12330	eval-mlogloss:1.20920
[20]	train-mlogloss:0.03118	eval-mlogloss:0.18994
[40]	train-mlogloss:0.02000	eval-mlogloss:0.16614
[50]	train-mlogloss:0.01899	eval-mlogloss:0.16915

Train Accuracy: 1.0000
Test Accuracy: 0.9455

All visualizations logged to W&B dashboard
Check the 'Media' tab in W&B for images
Check the 'Charts' tab for line plots

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        34
         1.0       0.93      0.78      0.85        18
         2.0       1.00      0.95      0.98        22
         3.0       0.88      0.93      0.90        15
         4.0       0.88      1.00      0.94        15
         5.0       0.86      1.00      0.92         6

    accuracy                           0.95       110
   macro avg       0.92      0.94      0.93       110
weighted avg       0.95      0.95     

best_iteration,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eval_loss,█▆▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁
train_accuracy,▁
train_loss,█▆▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_iteration,40
epoch,50
eval_loss,0.16915
test_accuracy,0.94545
train_accuracy,1
